In [2]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta

In [3]:
path = r"C:\Users\Admin\Downloads\RB6 0507 - 1207.csv"
data=pd.read_csv(path)

In [4]:
df = data.copy()

In [5]:
#cleaning data
def cleaning_data (df, day_begin, day_window, **args):
    #format day install
    df['Install_Day'] = pd.to_datetime(df['Install_Day'])
    #calculate range of install days
    day_range = pd.date_range(start = day_begin, periods = day_window, freq='1D')
    df.query('Install_Day in @day_range', inplace = True)
    df = df[df.Country_Code == args ['Country_Code']]
    try:
        data.query('Media_source in @args["Media_Source"]', inplace = True)
    except:
        pass
    return df

In [41]:
df_pivot = cleaning_data(df, day_begin = '2021-07-05', day_window = 5 , Country_Code ='BR' )
df_pivot

,Media_Source,Campaign,Adset,Ad,Country_Code,Install_Day,Site_ID,Keywords,App_Version,IS_Revenue_3,...,Ads_Revenue_3,Ads_Revenue_7,Ads_Revenue_14,IS_Impression_3,IS_Impression_7,IS_Impression_14,RV_Impression_3,RV_Impression_7,RV_Impression_14,User
9,Organic,NaN,NaN,NaN,BR,2021-07-05,NaN,NaN,1.9,0.170749,...,0.451451,0.466082,0.466082,38.0,40.0,40.0,41.0,42.0,42.0,20
20,Organic,NaN,NaN,NaN,BR,2021-07-07,NaN,NaN,1.9,0.307429,...,0.798466,0.817569,0.817569,149.0,150.0,150.0,121.0,131.0,131.0,20
21,Organic,NaN,NaN,NaN,BR,2021-07-07,NaN,NaN,1.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
22,Organic,NaN,NaN,NaN,BR,2021-07-06,NaN,NaN,1.9,0.385900,...,1.041741,1.154608,1.154608,114.0,119.0,119.0,134.0,149.0,149.0,20
23,Organic,NaN,NaN,NaN,BR,2021-07-06,NaN,NaN,1.7,0.015933,...,0.028537,0.028537,0.028537,3.0,3.0,3.0,2.0,2.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5047,unityads_int,Unity_Ball6_CPI_030321,UAC1_30s_E1,UAC1_30s_E1,BR,2021-07-08,es9F1sdE4xkv,NaN,1.9,0.008558,...,0.008558,0.008558,0.008558,2.0,2.0,2.0,0.0,0.0,0.0,1
5048,unityads_int,Unity_Ball6_CPI_030321,UAC5_E1_30s,UAC5_E1_30s,BR,2021-07-06,isLbfBwE176n,NaN,1.9,0.035770,...,0.067576,0.067576,0.067576,16.0,16.0,16.0,7.0,7.0,7.0,1
5049,unityads_int,Unity_Ball6_CPI_030321,UAC1_30s_E1,UAC1_30s_E1,BR,2021-07-09,gKz2DU1I9gzS,NaN,1.9,0.035579,...,0.064403,0.064403,0.064403,4.0,4.0,4.0,3.0,3.0,3.0,1
5052,unityads_int,Unity_Ball6_CPI_030321,UAC5_E1_30s,UAC5_E1_30s,BR,2021-07-09,6iO3tST1x89h,NaN,1.9,0.015126,...,0.028604,0.028604,0.028604,2.0,2.0,2.0,2.0,2.0,2.0,1


In [7]:
df_pivot['Install_Day'].unique()

array(['2021-07-05T00:00:00.000000000', '2021-07-07T00:00:00.000000000',
       '2021-07-06T00:00:00.000000000', '2021-07-08T00:00:00.000000000',
       '2021-07-09T00:00:00.000000000'], dtype='datetime64[ns]')

In [8]:
#calculate iUV3 and LTV
def ltv (df_pivot, day_begin = '2021-07-05', day_window = 5):
    #group by media source and campaign
    df_pivot = df_pivot.groupby(['Media_Source','Campaign','Keywords'], dropna = False).agg({'User':np.sum, 'Ads_Revenue_3':np.sum, 'IS_Impression_3':np.sum, 'RV_Impression_3':np.sum}).reset_index()
    #add columns and calculate values in added columns
    df_pivot['UV3'] = df_pivot['Ads_Revenue_3']/df_pivot['User']
    df_pivot['LTV'] = df_pivot['UV3']/0.9
    df_pivot['iUV14'] = df_pivot['LTV']/0.9
    df_pivot['IS/user'] = df_pivot['IS_Impression_3']/df_pivot['User']
    df_pivot['RV/user'] = df_pivot['RV_Impression_3']/df_pivot['User']
    #add grand total row, customize columns
    df_pivot=df_pivot.append(df_pivot[['User','Ads_Revenue_3','IS_Impression_3', 'RV_Impression_3']].sum().rename('Grand_Total'))
    #df_pivot = df_pivot.loc['Grand_Total', 'UV3':'RV/user']
    return df_pivot

In [42]:
LTV= ltv(df_pivot).fillna('')
LTV

,Media_Source,Campaign,Keywords,User,Ads_Revenue_3,IS_Impression_3,RV_Impression_3,UV3,LTV,iUV14,IS/user,RV/user
0,Organic,,,119.0,4.290565,566.0,607.0,0.036055,0.040061,0.044513,4.756303,5.10084
1,applovin_int,Applovin_ReB6i_260221,,1.0,0.144635,10.0,28.0,0.144635,0.160706,0.178562,10.0,28.0
2,mintegral_int,Ball6_240221,,290.0,22.996914,2053.0,3185.0,0.0793,0.088111,0.097901,7.07931,10.982759
3,unityads_int,Unity_Ball6_CPI_030321,,25.0,1.836950,136.0,160.0,0.073478,0.081642,0.090714,5.44,6.4
4,unityads_int,Unity_Ball6_CPI_030321_Fads,,34.0,2.052560,214.0,270.0,0.060369,0.067077,0.07453,6.294118,7.941176
Grand_Total,,,,469.0,31.321624,2979.0,4250.0,,,,,


In [10]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

In [11]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# here enter the id of your google sheet
SAMPLE_SPREADSHEET_ID_input = '18Fl333OEzDehkL6PVFWukGOeGeV9_a-fVQQ57k-vA5c'
SAMPLE_RANGE_NAME = 'A1:AA17000'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                r'C:\Users\Admin\Downloads\credentials.json', SCOPES) # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input:
        print('No data found.')


main()

No data found.


In [43]:
#change this by your sheet ID
SAMPLE_SPREADSHEET_ID_input = '18Fl333OEzDehkL6PVFWukGOeGeV9_a-fVQQ57k-vA5c'
# SAMPLE_SPREADSHEET_ID_input = '1Ui7R2PWov5yWHUqra5aKQBh2GFL-zVIliJaZg-eVVgQ'

#change the range if needed
SAMPLE_RANGE_NAME = 'BR!A1'

def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
    global service
    SCOPES = [scope for scope in scopes[0]]
    #print(SCOPES)
    
    cred = None

    if os.path.exists('token_write.pickle'):
        with open('token_write.pickle', 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
            cred = flow.run_local_server()

        with open('token_write.pickle', 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(api_service_name, api_version, credentials=cred)
        print(api_service_name, 'service created successfully')
        #return service
    except Exception as e:
        print(e)
        #return None
        
# change 'my_json_file.json' by your downloaded JSON file.
Create_Service(r"C:\Users\Admin\Downloads\credentials.json", 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])
    
def Export_Data_To_Sheets():
    response_date = service.spreadsheets().values().update(
        spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
        valueInputOption='RAW',
        range=SAMPLE_RANGE_NAME,
        body=dict(
            majorDimension='ROWS',
            values=LTV.T.reset_index().T.values.tolist())

    ).execute()
    print('Sheet successfully Updated')

Export_Data_To_Sheets()


sheets service created successfully
Sheet successfully Updated
